# 13 - Modeling Modules with Glass


This journal shows how to add glass to the modules. It also shows how to sample irradiance on the surface of the glass, as well as on the surface of the module. Surface of the module is slightlyt less irradiance due to fresnel losses (a.k.a. Angle of Incidence losses (AOI))

Some examples of the module with glass we'll create:

![Bottom Reflection](../images_wiki/Journal13Pics/Glass_bottom_reflection.PNG)
![Side closeup](../images_wiki/Journal13Pics/Glass_side_closeup.PNG)
![Tiled view](../images_wiki/Journal13Pics/Glass_tilted_reflection.PNG)


In [1]:
import os
from bifacial_radiance import *
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_13')
if not os.path.exists(testfolder):
    os.makedirs(testfolder)

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_13


In [2]:
demo = RadianceObj('tutorial_13',str(testfolder))  
demo.setGround(0.30)  # This prints available materials.
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(epwfile) 
demo.gendaylit(8)  # January 1 4pm (timepoint # 8)\


path = C:\Users\sayala\Documents\GitHub\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_13
Making path: images
Making path: objects
Making path: results
Making path: skies
Making path: EPWs
Making path: materials
Loading albedo, 1 value(s), 0.300 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


'skies\\sky2_37.5_-77.33_2021-01-01_1600.rad'

## Modeling example with glass

In [3]:
module_type = 'test-module' 

numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

visible = True
diameter = 0.15
tubetype = 'round'
material = 'Metal_Grey'
xgap = 0.1
ygap = 0
zgap = 0.05
numpanels = 1
axisofrotationTorqueTube = False
glass = True

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

mymodule = demo.makeModule(name=module_type, x=1, y=1, xgap=xgap, ygap=ygap, 
                           zgap=zgap, numpanels=numpanels, glass=glass, z=0.0002)
mymodule.addTorquetube(material=material, tubetype=tubetype, diameter=diameter,
                      axisofrotation=axisofrotationTorqueTube, recompile=False)
mymodule.addCellModule(**cellLevelModuleParams)

sceneDict = {'tilt':0,'pitch':5.5,'hub_height':1.0,'azimuth':90, 'nMods': 20, 'nRows': 1, 'originx':0, 'originy':0} 
scene = demo.makeScene(module_type, sceneDict)
octfile = demo.makeOct(demo.getfilelist())  


Module Name: test-module
Module test-module updated in module.json
Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

Created tutorial_13.oct


## Advanced Rendering:

The images at the beginning of the journal can be made pretty with advanced rendering. This is the workflow:

    rvu -> rpict -> pcond -> pfilt -> ra_tiff -> convert

In detail:

***1. Use rvu to view the oct file***

    rvu 1axis_07_01_08.oct 

Use aim and origin to move around, zoom in/out, etc. Save a view file with view render.

***2. Run rpict to render the image to hdr***

This is the time consuming step. It takes between 1 and 3 hours depending on how complex the geometry is.

    rpict -x 4800 -y 4800 -i -ps 1 -dp 530 -ar 964 -ds 0.016 -dj 1 -dt 0.03 -dc 0.9 -dr 5 -st 0.12 -ab 5 -aa 0.11 -ad 5800 -as 5800 -av 25 25 25 -lr 14 -lw 0.0002 -vf render.vf bifacial_example.oct > render.hdr


***3. Run pcond to mimic human visual response***

    pcond -h render.hdr > render.pcond.hdr


***4. Resize and adjust exposure with pfilt***

    pfilt -e +0.2 -x /4 -y /4 render.pcond.hdr > render.pcond.pfilt.hdr


***5. Convert hdr to tif***

    ra_tiff render.pcond.pfilt.hdr render.tif


***6. Convert tif to png with imagemagick convert utility***

    convert render.tif render.png


***7. Annotate the image with convert***

    convert render.png -fill black -gravity South -annotate +0+5 'Created with NREL bifacial_radiance https://github.com/NREL/bifacial_radiance' render_annotated.png


In [5]:
analysis = AnalysisObj(octfile, demo.basename)

## Scanning Outside of the module, the surface of the glass

In [7]:
frontscan, backscan = analysis.moduleAnalysis(scene)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  
load.read1Result('results\irr_tutorial_13.csv')

Linescan in process: tutorial_13_Front
Linescan in process: tutorial_13_Back
Saved: results\irr_tutorial_13.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.758906,4.646956e-17,0.647441,0.645447,a9.4.a2.1.0.cellPVmodule.6457,a9.4.a2.1.0.cellPVmodule.2310,62.747280,14.482907,0.230810
1,0.569306,3.485993e-17,0.735852,0.733858,a9.4.a2.2.0.cellPVmodule.6457,a9.4.a2.2.0.cellPVmodule.2310,62.956283,13.176527,0.209293
2,0.379706,2.325031e-17,0.824264,0.822270,a9.4.a2.3.0.cellPVmodule.6457,a9.4.a2.3.0.cellPVmodule.2310,63.080187,12.902470,0.204538
3,0.190107,1.164068e-17,0.912676,0.910682,a9.4.a2.4.0.cellPVmodule.6457,a9.4.a2.4.0.cellPVmodule.2310,63.165527,12.923240,0.204590
4,0.000507,3.105349e-20,1.001088,0.999094,a9.4.a0.test-module_Glass.2310,a9.4.a0.test-module_Glass.6457,15.096727,25.684550,1.701220
5,-0.189092,-1.157857e-17,1.089499,1.087505,a9.4.a2.7.0.cellPVmodule.6457,a9.4.a2.7.0.cellPVmodule.2310,63.728150,17.160590,0.269274
6,-0.378692,-2.318820e-17,1.177911,1.175917,a9.4.a2.8.0.cellPVmodule.6457,a9.4.a2.8.0.cellPVmodule.2310,63.839600,16.670997,0.261135
7,-0.568292,-3.479783e-17,1.266323,1.264329,a9.4.a2.9.0.cellPVmodule.6457,a9.4.a2.9.0.cellPVmodule.2310,63.949897,17.361977,0.271489
8,-0.757891,-4.640745e-17,1.354735,1.352741,a9.4.a2.10.0.cellPVmodule.6457,a9.4.a2.10.0.cellPVmodule.2310,65.141047,18.096150,0.277795


## Scanning Inside of the module, the surface of the cells

In [10]:
frontscan, backscan = analysis.moduleAnalysis(scene, frontsurfaceoffset=0.05, backsurfaceoffset = 0.05)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  
load.read1Result('results\irr_tutorial_13.csv')

Linescan in process: tutorial_13_Front
Linescan in process: tutorial_13_Back
Saved: results\irr_tutorial_13.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.779614,4.773758e-17,0.691850,0.601038,a9.4.a0.test-module_Glass.6457,a9.4.a2.1.0.cellPVmodule.2310,62.707750,14.261603,0.227426
1,0.590014,3.612795e-17,0.780261,0.689449,a9.4.a0.test-module_Glass.6457,a9.4.a2.2.0.cellPVmodule.2310,62.878220,12.879707,0.204832
2,0.400415,2.451832e-17,0.868673,0.777861,a9.4.a0.test-module_Glass.6457,a9.4.a2.3.0.cellPVmodule.2310,63.047787,12.792097,0.202892
3,0.210815,1.290870e-17,0.957085,0.866273,a9.4.a0.test-module_Glass.6457,a9.4.a2.4.0.cellPVmodule.2310,63.250300,12.453710,0.196893
4,0.021215,1.299071e-18,1.045497,0.954685,a9.4.a0.test-module_Glass.6457,a9.4.a0.test-module_Glass.6457,14.642467,25.684550,1.753994
5,-0.168384,-1.031056e-17,1.133908,1.043096,a9.4.a0.test-module_Glass.6457,a9.4.a2.7.0.cellPVmodule.2310,63.567113,16.889767,0.265696
6,-0.357984,-2.192018e-17,1.222320,1.131508,a9.4.a0.test-module_Glass.6457,a9.4.a2.8.0.cellPVmodule.2310,63.962993,16.568733,0.259032
7,-0.547583,-3.352981e-17,1.310732,1.219920,a9.4.a0.test-module_Glass.6457,a9.4.a2.9.0.cellPVmodule.2310,64.008537,17.348283,0.271027
8,-0.737183,-4.513944e-17,1.399144,1.308332,a9.4.a0.test-module_Glass.6457,a9.4.a2.10.0.cellPVmodule.2310,65.121143,18.815253,0.288923
